In [33]:
from openai import OpenAI
from pathlib import Path
import os
import time


# print(API_KEY)
client = OpenAI(api_key=API_KEY)

In [35]:
my_assistant = client.beta.assistants.create(
    instructions="Ты находишься в игре, связанной с экономикой. Когда я пишу тебе запрос, ты должен генерировать интересную задачу по макроэкономике (используя macro1_lect_1_30.pdf)",
    name="Macro Tutor",
    tools=[{"type": "file_search"}],
    model="gpt-3.5-turbo-0125"
)
my_assistant

Assistant(id='asst_cglpEtrf39z1AAzdAHw4MbzK', created_at=1716246697, description=None, instructions='Ты находишься в игре, связанной с экономикой. Когда я пишу тебе запрос, ты должен генерировать интересную задачу по макроэкономике (используя macro1_lect_1_30.pdf)', metadata={}, model='gpt-3.5-turbo-0125', name='Macro Tutor', object='assistant', tools=[FileSearchTool(type='file_search')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)

In [36]:

vector_store = client.beta.vector_stores.create(name="Macro1 Store")


In [108]:
file_path_gol = ["/Applications/MAMP/htdocs/GOL_WEB/PHP_AI/fin_rin.pdf"]
file_stream_gol = [open(path, "rb") for path in file_path_gol]

In [109]:
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_stream_gol
)

In [105]:
# file_batch.status
file_batch.file_counts

FileCounts(cancelled=0, completed=0, failed=1, in_progress=0, total=1)

In [110]:
client.beta.vector_stores.retrieve(vector_store_id=vector_store.id)

VectorStore(id='vs_9KCNuQDguJWfwRAwtVh9P7Rl', created_at=1716246702, file_counts=FileCounts(cancelled=0, completed=2, failed=1, in_progress=0, total=3), last_active_at=1716410097, metadata={}, name='Macro1 Store', object='vector_store', status='completed', usage_bytes=925943, expires_after=None, expires_at=None)

In [113]:
vector_store

VectorStore(id='vs_9KCNuQDguJWfwRAwtVh9P7Rl', created_at=1716246702, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1716246702, metadata={}, name='Macro1 Store', object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None)

In [114]:
my_assistant = client.beta.assistants.update(
  assistant_id=my_assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [115]:
thread = client.beta.threads.create(
      messages=[
    {
      "role": "user",
      "content": "Сгенерируй раунд для моей игры Game Of Life. Пусть герою 12 лет и думает, как можно заработать",
      # Attach the new file to the message.
    }
  ]
)



In [72]:
# # Use the create and poll SDK helper to create a run and poll the status of
# # the run until it's in a terminal state.

# run = client.beta.threads.runs.create_and_poll(
#     thread_id=thread.id, assistant_id=my_assistant.id
# )
# # print(run)
# messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
# # print(messages)
# message_content = messages[0].content[0].text
# print(message_content)
# annotations = message_content['annotations']
# citations = []
# for index, annotation in enumerate(annotations):
#     message_content['value'] = message_content['value'].replace(annotation['text'], f"[{index}]")
#     if file_citation := getattr(annotation, "file_citation", None):
#         cited_file = client.files.retrieve(file_citation.file_id)
#         citations.append(f"[{index}] {cited_file.filename}")

# print(message_content['value'])
# print("\n".join(citations))

{'value': "Here is a macroeconomics problem related to the IS-LM-BP model for your Game Of Life WEB:\n\nImagine in the Game Of Life WEB that country A is facing a situation where there is a sudden surge in capital outflows due to a decrease in investor confidence. Analyze this scenario using the IS-LM-BP model:\n- Describe the effects of the capital outflows on country A's balance of payments and exchange rate.\n- Illustrate how the IS-LM-BP model would adjust to accommodate the change in capital flows.\n- Discuss the implications of these adjustments on country A's output level and interest rates in both the short and long term.\n\nThis problem requires applying the concepts of the IS-LM-BP model to analyze the impact of capital outflows on an open economy like country A in the Game Of Life WEB, considering the interplay between macroeconomic variables such as exchange rates, interest rates, and output levels.【4:0†source】【4:1†source】【4:3†source】.", 'annotations': [{'type': 'file_citat

In [116]:
def chat_with_gpt(thread_new):
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread_new.id, assistant_id=my_assistant.id
    )

    messages = list(client.beta.threads.messages.list(thread_id=thread_new.id, run_id=run.id))
    # print(messages)
    from_user_msg = input('NEXT:')
    while from_user_msg != 'STOP':
        
        client.beta.threads.messages.create(
            thread_id=thread_new.id, role='user', content=from_user_msg
        )
        
        client.beta.threads.update(thread_id=thread_new.id)
        client.beta.threads.runs.update(run_id=run.id, thread_id=thread_new.id)
        messages = list(client.beta.threads.messages.list(thread_id=thread_new.id, run_id=run.id))
        message_content = messages[0].content[0].text
        print(message_content)
        annotations = message_content['annotations']
        citations = []
        for index, annotation in enumerate(annotations):
            message_content['value'] = message_content['value'].replace(annotation['text'], f"[{index}]")
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        # print(message_content['value'])
        
        from_user_msg = input('NEXT:')
    # print("\n".join(citations))

In [123]:
empty_thread = client.beta.threads.create(tool_resources={
    "file_search": {
        "vector_stores": {
            "file_ids" : ["file-U4RK7QEp6rtEOcLkjK1HI4Cd"]}}},)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'tool_resources.file_search.vector_stores[0]': expected an object, but got a string instead.", 'type': 'invalid_request_error', 'param': 'tool_resources.file_search.vector_stores[0]', 'code': 'invalid_type'}}

In [117]:
chat_with_gpt(thread)

{'value': 'For your Game of Life round, you can challenge the 12-year-old character to think about ways to earn money by exploring the concept of the business cycle in macroeconomics. The character could learn about different phases of the business cycle such as recession, crisis, upturn, and peak, and how different economic variables behave during these phases (e.g., cyclical, countercyclical, or acyclical variables). This knowledge could help the character make informed decisions on how to earn money based on the current economic conditions. \n\nYou can introduce scenarios where the character has to adapt their earning strategies based on the phase of the business cycle and explore how their decisions impact their financial well-being in the game.\n\nCitation: Presentation on macroeconomics concepts from macro1_lect_1_30.pdf【4:1†source】【4:2†source】.', 'annotations': [{'type': 'file_citation', 'text': '【4:1†source】', 'start_index': 823, 'end_index': 835, 'file_citation': {'file_id': '

In [120]:
client.beta.threads.messages.list(thread_id=thread.id)

SyncCursorPage[Message](data=[Message(id='msg_hLWjBbSXiec8P3we7QkTN74D', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='сгенерируй задачу по теме IS-LM-BP'), type='text')], created_at=1716410591, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_ourk94lO4U2spKGPetV851vY'), Message(id='msg_AWx9i0UJnt11Tvd7Amv9uTJp', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Отличн. Теперь герою 18 лет он задумывается над тем, чтобы инвестировать в фондовый рынок. Сгенерируй раунд, где ему потребуется правильно распределить свой капитал, чтобы максимизировать прибыль. В следующем сообщении дождись ответа от игрока и дай ему оценку'), type='text')], created_at=1716410518, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, sta

In [130]:
new_run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=my_assistant.id,
    instructions="Пожалуйста, генерируй тексты задания по заданным пользователем темам"
)

In [165]:
thread = client.beta.threads.create()

In [166]:
import time
def send_message_and_get_response(thread_id, assistant_id, message):
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions='Пожалуйста, генерируй тексты заданий по заданным пользователем темам'
    )
    client.beta.threads.messages.create(thread_id=thread_id, content=message, role='user')
    # while client.beta.threads.messages.list(thread_id=thread.id).data[0].content[0].text.value == message:
    #     pass
    # time.sleep(10)
    return client.beta.threads.messages.list(thread_id=thread.id).data[0].content[0].text.value
    # return run['data']['choices'][0]['message']['content']

# Получаем и отправляем сообщения в цикле
usr_msg = input("Введите ваше сообщение (или 0 для выхода): ")
while usr_msg != '0':
    response = send_message_and_get_response(thread.id, my_assistant.id, usr_msg)
    print("Ответ ассистента:", response)
    usr_msg = input("Введите ваше сообщение (или 0 для выхода): ")

Ответ ассистента: составь задачу по теме IS-LM-BP. Необходимо, чтобы там были числа


In [167]:
client.beta.threads.messages.list(thread_id=thread.id)

SyncCursorPage[Message](data=[Message(id='msg_0YsMzNTdcbX7trruFfQGPHSR', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='составь задачу по теме IS-LM-BP. Необходимо, чтобы там были числа'), type='text')], created_at=1716464421, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_mxXMKgUg8vEzADBDfL2F3pqi'), Message(id='msg_ROsH2wI8XdtexpdU65I8KU8X', assistant_id='asst_cglpEtrf39z1AAzdAHw4MbzK', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="How can I assist you with the uploaded files today? If you need me to search for specific information, please let me know the topics or queries you're interested in!"), type='text')], created_at=1716464419, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_NHqYv2LqWXxUr6ZTWl4uXMxI', status=No

In [170]:
new_assistant = client.beta.assistants.create(
    name='Фин.рынки Task Creator',
    description='Ты - генератор раундов для игры Game Of Life (смотри в GameOfLife.pdf).',
    instructions='Ты должен генерировать задания, связанные с экономикой (смотри в macro1_lect_1_30.pdf). Твои задания могут быть как теоретические, так и вычислительные',
    model='gpt-3.5-turbo-0125',
    tool_resources={
        "file_search" : {
            "vector_store_ids" : ['vs_9KCNuQDguJWfwRAwtVh9P7Rl']
        }
    },
    # response_format={"type" : "json_object"}
)

In [183]:
batch_add = client.beta.vector_stores.file_batches.create(
  vector_store_id="vs_9KCNuQDguJWfwRAwtVh9P7Rl",
  file_ids=["file-vd21YVg1jAjlsLBTGRp6PBUw", "file-IwozwGNhnRkCa2foJuChW78D"]
)
time.sleep(10)
print(batch_add.status)

in_progress


In [203]:
print(batch_add.status)

in_progress


In [180]:
another_thread = client.beta.threads.create(
    messages=[
        {
            "role" : "user",
            "content" : "Сгенерируй новый раунд. Моему герою сейчас 18 лет. Он начал задумываться о первом заработке",
            "attachments" : [
                {
                    "file_id" : "file-IwozwGNhnRkCa2foJuChW78D",
                    "tools" : [{"type" : "file_search"}]
                }
            ]
        }
    ]
)

In [205]:
client.beta.vector_stores.retrieve(vector_store_id=vector_store.id)

VectorStore(id='vs_9KCNuQDguJWfwRAwtVh9P7Rl', created_at=1716246702, file_counts=FileCounts(cancelled=0, completed=4, failed=1, in_progress=0, total=5), last_active_at=1716488099, metadata={}, name='Macro1 Store', object='vector_store', status='completed', usage_bytes=1105074, expires_after=None, expires_at=None)

In [206]:
def create_msg(user_msg):
    print(f"Твоё сообщение: {user_msg}")
    message_create = client.beta.threads.messages.create(
        role="user",
        thread_id=another_thread.id,
        content=f"{user_msg}"
    )
    print(f"Что создалось: {message_create}")
    
def thread_running():
    run = client.beta.threads.runs.create(
        thread_id=another_thread.id,
        assistant_id=new_assistant.id
    )
    return run.id

def refresh_thread(run_id):
    run_info = client.beta.threads.runs.retrieve(
        thread_id=another_thread.id,
        run_id=run_id
    )
    
    
def get_last_msg():
    messages = client.beta.threads.messages.list(
        thread_id=another_thread.id
    )
    return messages


def ask_assistant(user_msg):
    create_msg(user_msg)
    run_id = thread_running()
    time.sleep(15)
    
    refresh_thread(run_id)
    
    messages = get_last_msg()
    
    # try:
    answer = messages.data[0].content[0].text.value
    # except:
        
    print(answer)
    
    

while True:
    print("Введите запрос:")
    
    user_msg = input()
    
    ask_assistant(f"{user_msg}")

Введите запрос:
Твоё сообщение: Сгенерируй задачу, связанную с инвестированием во фьючерсы
Что создалось: Message(id='msg_Yxo4ZE9t9bqnpNDOwl7efdBZ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Сгенерируй задачу, связанную с инвестированием во фьючерсы'), type='text')], created_at=1716490925, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_ddUPygmJ8frnq59NclcfB4Wp')
Вот задача по инвестированию во фьючерсы для тебя:

Допустим, инвестор решил инвестировать свои средства в фьючерсный контракт на нефть. Пусть стоимость одного фьючерсного контракта на нефть равна $50 за баррель. Инвестор решает купить 10 фьючерсных контрактов на нефть.

Дано:
- Стоимость одного фьючерсного контракта на нефть: $50 за баррель.
- Количество фьючерсных контрактов, которые инвестор решил купить: 10 контрактов.

Задача:
1. Рассчитай общую стоимость покупк

BadRequestError: Error code: 400 - {'error': {'message': 'Message content must be non-empty.', 'type': 'invalid_request_error', 'param': 'content', 'code': None}}

In [182]:
client.beta.threads.messages.list(thread_id=another_thread.id)

SyncCursorPage[Message](data=[Message(id='msg_QbQJjc0MZyZuinVpDP3CYuo5', assistant_id='asst_2zXy7YsxfOBl4WVKm73h3xqo', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Конечно, вот задача с HTML-тегами для удобной вставки на сайт:\n\n```html\n<h3>Задача по модели IS-LM-BP</h3>\n<p>Представим экономику, которая находится в равновесии при данном уровне государственных закупок и процентной ставке. Рассмотрим модель IS-LM-BP.</p>\n<p><strong>Дано:</strong></p>\n<ul>\n  <li>Исходный уровень государственных закупок (G) и объём облигаций в обращении (B).</li>\n  <li>Первоначальный уровень процентной ставки (r).</li>\n</ul>\n<p><strong>Шок:</strong><br>\nУровень государственных закупок (G) увеличился за счёт выпуска дополнительных облигаций.</p>\n<p><strong>Задача:</strong></p>\n<ol>\n  <li>Объясни, как повлияет увеличение уровня государственных закупок на протекание процессов в экономике.</li>\n  <li>Определи, как изменятся следующие параметры в ре